In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import random
random.seed(42)

%load_ext autoreload
%autoreload 2

In [2]:
artembev = pd.read_excel("./1Artembev_PhC_sport_and_tripms.xls")
prokopbev = pd.read_excel('./2Prokopbev_PhC_sport_and_tripms.xls')
volkov = pd.read_excel('./3Volkov_PhC_sport_and_tripms.xls')

In [39]:
artembev.columns
art_key_features = artembev.drop(columns=['day number', 'Speed','Heart rate','Load','Avg PhC 9days', 'Avg PhC 3days',
                                        'Unnamed: 13','Unnamed: 14','Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 
                        'Unnamed: 19', 'Unnamed: 8'], axis=0)
art_key_features.drop(labels=range(11), inplace=True)
art_forecast = art_key_features['Forecast']
art_key_features.drop(columns=['Forecast'], inplace=True)
art_key_features.head()

,Physiological Cost (PhC),TRIMP1,TRIMP2,TRIMP3,TRIMP4
11,0.411155,802.127,138.638,1.23169,0
12,0.395014,1450.13,100.318,1341.57,579.98
13,0.392351,732.626,70.6103,2440.17,1695.58
14,NaN,809.691,13.2364,1778.61,880.455
15,0.408444,769.12,26.4721,1773.54,887.856


In [48]:
x = art_key_features.shape[0] - 3
art_key_features["TRIMP1"].iloc[0-1:2]

nan

In [49]:
%%time
for col in art_key_features.columns:
    art_key_features['Mean '+col]=np.zeros(art_key_features.shape[0])
    art_key_features.loc[:,'Mean '+col].iloc[0] = art_key_features[col].iloc[0]
    art_key_features.loc[:,'Mean '+col].iloc[1] = art_key_features[col].iloc[0:2].mean()
    for i in range(art_key_features.shape[0]-3):
        art_key_features.loc[:,'Mean '+col].iloc[i+2] = art_key_features[col].iloc[i:i+3].mean()

CPU times: user 2min 21s, sys: 23.4 ms, total: 2min 21s
Wall time: 2min 21s


In [50]:
art_key_features.head()

,Physiological Cost (PhC),TRIMP1,TRIMP2,TRIMP3,TRIMP4,Mean Physiological Cost (PhC),Mean TRIMP1,Mean TRIMP2,Mean TRIMP3,Mean TRIMP4
11,0.411155,802.127,138.638,1.23169,0,0.411155,802.126507,138.638081,1.231694,0.000000
12,0.395014,1450.13,100.318,1341.57,579.98,0.403084,1126.128154,119.477807,671.398724,289.989948
13,0.392351,732.626,70.6103,2440.17,1695.58,0.399507,994.960748,103.188621,1260.987536,758.520146
14,NaN,809.691,13.2364,1778.61,880.455,0.393683,997.482226,61.388053,1853.448052,1052.005165
15,0.408444,769.12,26.4721,1773.54,887.856,0.400398,770.478993,36.772901,1997.440934,1154.630564


In [52]:
means = art_key_features.drop(columns=[x for x in art_key_features.columns if x.startswith('TRIMP')])
means.head()

,Physiological Cost (PhC),Mean Physiological Cost (PhC),Mean TRIMP1,Mean TRIMP2,Mean TRIMP3,Mean TRIMP4
11,0.411155,0.411155,802.126507,138.638081,1.231694,0.000000
12,0.395014,0.403084,1126.128154,119.477807,671.398724,289.989948
13,0.392351,0.399507,994.960748,103.188621,1260.987536,758.520146
14,NaN,0.393683,997.482226,61.388053,1853.448052,1052.005165
15,0.408444,0.400398,770.478993,36.772901,1997.440934,1154.630564


Для начала оставим дерево без ограничений по глубине

In [ ]:
tree = DecisionTreeRegressor()
X = means.drop(columns=['Mean Physiological Cost (PhC)'])
y = means['Mean Physiological Cost (PhC)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)